classifiers: nn.crossentropyloss = -log-likelihood --- use for logp(y) and -logq(y/x) for support <br>
kl-div: <br>
reconstr-loss: set reduction to none and then take mean of losses per image in the total batch. This gives reconstr-loss per image for further computation<br> 

In [16]:
import sys
import random
import numpy as np
import torch
import learn2learn as l2l
import pandas as pd

from torch import nn, optim
from torch.nn import functional as F
import torchvision
from torchvision import datasets, transforms
from PIL.Image import LANCZOS
from config import *

import json
import copy

from torch.distributions.multivariate_normal import MultivariateNormal


In [2]:
torch.cuda.is_available()


True

In [3]:
sys.path.append('/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn')
sys.path

['/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn/src',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2021.8.1236758218/pythonFiles',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2021.8.1236758218/pythonFiles/lib/python',
 '/home/anuj/anaconda3/envs/torch/lib/python38.zip',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/lib-dynload',
 '',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/datasets-1.2.1-py3.8.egg',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/locket-0.2.1-py3.8.egg',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/IPython/extensions',
 '/home/anuj/.ipython',
 '/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn']

In [4]:
from data.loaders import Omniglotmix, MiniImageNet
from data.taskers import gen_tasks
from src.zoo.archs import CVAE
from src.zoo.lpo_utils import proto_distr, classify, set_sets, accuracy, inner_adapt_lpo

#from src.utils import Profiler

In [5]:
train_tasks = gen_tasks(dataname='miniimagenet', root='../../mini_imagenet', mode='train', n_ways=5, k_shots=3, q_shots=10, image_transforms=None)

In [6]:
classes = list(range(1623))
random.shuffle(classes)
image_transforms = transforms.Compose([transforms.Resize(28, interpolation=LANCZOS),
                                                    transforms.ToTensor(),
                                                    lambda x: 1.0 - x,
                                                ])
train_tasks = gen_tasks('omniglot', '/home/anuj/Desktop/Work/TU_Delft/research/implement/omniglot', image_transforms=image_transforms, n_ways=5, k_shots=3, q_shots=10, classes=classes[:1100])

/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/torchvision/transforms/transforms.py:257: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [7]:
ttask = train_tasks.sample()

In [13]:
loss = nn.BCELoss(reduction='none')
learner = CVAE(in_channels=1, y_shape=5, base_channels=32, latent_dim=64).to('cuda')
support, y_support, queries, qs, y_queries, queries_labels = set_sets(ttask, 5, 3, 10, 'cuda')
# Running inner adaptation loop
J_alpha, query_preds = inner_adapt_lpo(support, y_support, qs, y_queries, learner, loss, 5,3,10)

In [17]:
learner_temp = copy.deepcopy(learner.state_dict())

In [14]:
query_preds[::5,]

tensor([[0.1999, 0.2000, 0.1999, 0.2002, 0.2000],
        [0.1999, 0.2000, 0.1999, 0.2002, 0.2000],
        [0.1999, 0.2000, 0.1999, 0.2002, 0.2000],
        [0.1999, 0.2000, 0.1999, 0.2002, 0.2000],
        [0.1999, 0.2000, 0.1999, 0.2002, 0.2000],
        [0.1999, 0.2000, 0.1999, 0.2002, 0.2000],
        [0.1999, 0.2000, 0.1999, 0.2002, 0.2000],
        [0.1999, 0.2000, 0.1999, 0.2002, 0.2000],
        [0.1999, 0.2000, 0.1999, 0.2002, 0.2000],
        [0.1999, 0.2000, 0.1999, 0.2002, 0.2000],
        [0.1999, 0.2000, 0.1999, 0.2002, 0.2000],
        [0.1999, 0.2000, 0.1999, 0.2002, 0.2000],
        [0.1999, 0.2000, 0.1999, 0.2002, 0.2000],
        [0.1999, 0.2000, 0.1999, 0.2002, 0.2000],
        [0.1999, 0.2000, 0.1999, 0.2002, 0.2000],
        [0.1999, 0.2000, 0.1999, 0.2002, 0.2000],
        [0.1999, 0.2000, 0.1999, 0.2002, 0.2000],
        [0.1999, 0.2000, 0.1999, 0.2002, 0.2000],
        [0.1999, 0.2000, 0.1999, 0.2002, 0.2000],
        [0.1999, 0.2000, 0.1999, 0.2002, 0.2000],


In [12]:
accuracy(query_preds[::5,], queries_labels)

tensor(0.2000, device='cuda:0')

In [17]:
class Encoder(nn.Module):

    def __init__(self,
                 num_input_channels : int,
                 base_channel_size : int,
                 latent_dim : int,
                 act_fn : object = nn.GELU):
        """
        Inputs:
            - num_input_channels : Number of input channels of the image. For CIFAR, this parameter is 3
            - base_channel_size : Number of channels we use in the first convolutional layers. Deeper layers might use a duplicate of it.
            - latent_dim : Dimensionality of latent representation z
            - act_fn : Activation function used throughout the encoder network
        """
        super().__init__()
        c_hid = base_channel_size
        self.net = nn.Sequential(
            nn.Conv2d(num_input_channels, c_hid, kernel_size=3, padding=1, stride=2), # 28x28 => 16x16
            act_fn(),
            nn.Conv2d(c_hid, c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(c_hid, 2*c_hid, kernel_size=3, padding=1, stride=2), # 16x16 => 8x8
            act_fn(),
            nn.Conv2d(2*c_hid, 2*c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(2*c_hid, 2*c_hid, kernel_size=3, padding=1, stride=2), # 8x8 => 4x4
            act_fn(),
            nn.Flatten(), # Image grid to single feature vector
        )
        self.h1 = nn.Linear(2*16*c_hid, latent_dim)
        self.h2 = nn.Linear(2*16*c_hid, latent_dim)

    def forward(self, x):
        x = self.net(x)
        mu = self.h1(x)
        log_var = self.h2(x)
        return mu, log_var

class Decoder(nn.Module):

    def __init__(self,
                 num_input_channels : int,
                 base_channel_size : int,
                 latent_dim : int,
                 act_fn : object = nn.GELU):
        """
        Inputs:
            - num_input_channels : Number of channels of the image to reconstruct. For CIFAR, this parameter is 3
            - base_channel_size : Number of channels we use in the last convolutional layers. Early layers might use a duplicate of it.
            - latent_dim : Dimensionality of latent representation z + Dimensionality of one-hot encoded label 
            - act_fn : Activation function used throughout the decoder network
        """
        super().__init__()
        c_hid = base_channel_size
        self.linear = nn.Sequential(
            nn.Linear(latent_dim, 2*16*c_hid),
            act_fn()
        )
        self.net = nn.Sequential(
            nn.ConvTranspose2d(2*c_hid, 2*c_hid, kernel_size=3, padding=1, stride=2), # 4x4 => 8x8
            act_fn(),
            nn.Conv2d(2*c_hid, 2*c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.ConvTranspose2d(2*c_hid, c_hid, kernel_size=3, output_padding=1, padding=1, stride=2), # 8x8 => 16x16
            act_fn(),
            nn.Conv2d(c_hid, c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.ConvTranspose2d(c_hid, num_input_channels, kernel_size=3, output_padding=1, padding=1, stride=2), # 16x16 => 32x32
            nn.Sigmoid() # The input images is scaled between -1 and 1, hence the output has to be bounded as well
        )

    def forward(self, x):
        x = self.linear(x)
        x = x.reshape(x.shape[0], -1, 4, 4)
        x = self.net(x)
        return x

class CVAE(nn.Module):
    def __init__(self, in_channels, y_shape, base_channels=64, latent_dim=64):
        super(CVAE, self).__init__()
        self.in_channels = in_channels
        self.base_channels = base_channels
        self.latent_dim = latent_dim
        self.dec_latent_dim = self.latent_dim + y_shape

        self.encoder = Encoder(num_input_channels=self.in_channels, base_channel_size=self.base_channels, latent_dim=self.latent_dim)

        self.decoder = Decoder(num_input_channels=self.in_channels, base_channel_size=self.base_channels, latent_dim=self.dec_latent_dim)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5 * logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def forward(self, x, y):
        mu, log_var = self.encoder(x)
        z = self.reparameterize(mu, log_var)
        x = self.decoder(torch.cat([z, y], dim=1))
        return x, mu, log_var

In [20]:
cvae = CVAE(1, 5, 16, 64)
s_cap, mu_s, log_var_s = cvae(support, y_support)
mu_s.shape

torch.Size([15, 64])

In [8]:
support, y_support, queries, qs, y_queries =set_sets(ttask, 5, 3, 10, 'cpu')

In [11]:
enc = Encoder(1, 16, 64)
a = enc(support)
a[0].shape

torch.Size([15, 64])

In [12]:
def reparameterize(mu, logvar):
    std = torch.exp(0.5 * logvar)
    eps = torch.randn_like(std)
    return eps.mul(std).add_(mu)

z = reparameterize(a[0], a[1])
z.shape        

torch.Size([15, 64])

In [14]:
torch.cat([z, y_support], dim=1).shape

torch.Size([15, 69])

In [6]:
model = VAE(3, 16, 3, 3)
model.weight_init(mean=0, std=0.02)
model.to('cpu')

VAE(
  (conv1): Conv2d(3, 3, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv1_bn): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(3, 6, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv2_bn): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(6, 12, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv3_bn): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=1452, out_features=16, bias=True)
  (fc2): Linear(in_features=1452, out_features=16, bias=True)
  (d1): Linear(in_features=16, out_features=1452, bias=True)
  (deconv2): ConvTranspose2d(12, 6, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (deconv2_bn): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (deconv3): ConvTranspose2d(6, 3, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (deconv3_bn): BatchNorm2d(3, eps=1

In [7]:
def inner_adapt_lpo(task, n_ways, k_shots, q_shots, device):
    data, labels = task
    data, labels = data.to(device), labels.to(device)
    #data, labels = data.squeeze(0), labels.squeeze(0)
    sort = torch.sort(labels)
    data = data.squeeze(0)[sort.indices].squeeze(0)
    labels = labels.squeeze(0)[sort.indices].squeeze(0)
    total = n_ways * (k_shots + q_shots)
    queries_index = np.zeros(total)

    # Extracting the query datums from the entire task set
    for offset in range(n_ways):
        queries_index[np.random.choice(
            k_shots+q_shots, q_shots, replace=False) + ((k_shots + q_shots)*offset)] = True
    support = data[np.where(queries_index == 0)]
    support_labels = labels[np.where(queries_index == 0)]
    queries = data[np.where(queries_index == 1)]
    queries_labels = labels[np.where(queries_index == 1)]

    # Forward pass on the Support datums
    y_support = F.one_hot(support_labels, num_classes=n_ways)
    #support_cap, support_mu, support_log_var = learner(support, y_support)
    #proto_mu, proto_var = proto_distr(support_mu, support_log_var)

    # Forward pass on the Query datums
    y_queries = torch.tensor(range(n_ways))
    y_queries = y_queries.repeat(n_ways*q_shots)
    y_queries = F.one_hot(y_queries, num_classes=n_ways)
    qs = queries.repeat_interleave(n_ways, dim=0)
    #queries_cap, queries_mu, queries_log_var = learner(queries, y_queries)
    return support, y_support, queries, qs, y_queries 


In [8]:
a,b,c,d,e = inner_adapt_lpo(train_tasks.sample(), 5, 3, 10, 'cpu')

In [9]:
e.shape

torch.Size([250, 5])

In [10]:
r_s, mu_s, log_var_s = model(a)

In [11]:
mu_p, var_p = proto_distr(mu_s, log_var_s, 5, 3, 'average')

In [12]:
logits_s = classify(mu_p, var_p, mu_s)

In [130]:
r_q, mu_q, log_var_q = model(d)
logits_q = classify(mu_p, var_p, mu_q)

In [131]:
logits_q

tensor([[-5.9524, -5.4496, -4.9405, -8.4811, -5.8921],
        [-5.9524, -5.4496, -4.9405, -8.4811, -5.8921],
        [-5.9524, -5.4496, -4.9405, -8.4811, -5.8921],
        ...,
        [-4.5103, -2.3777, -3.4718, -5.9384, -4.3534],
        [-4.5103, -2.3777, -3.4718, -5.9384, -4.3534],
        [-4.5103, -2.3777, -3.4718, -5.9384, -4.3534]], grad_fn=<SubBackward0>)

In [132]:
logits_q.shape

torch.Size([250, 5])

In [35]:
classication_loss_q = torch.sum(e * torch.log(F.softmax(logits_q, dim=1)), dim=1).mean()
classication_loss_q

tensor(-3.6833, grad_fn=<MeanBackward0>)

In [34]:
loss = nn.CrossEntropyLoss()
classication_loss_q = loss(logits_q, torch.argmax(e, dim=1))
classication_loss_q

tensor(3.6833, grad_fn=<NllLossBackward>)

Keep cross-entropy loss from torch as is (just add alpha*celoss to other losses)

In [37]:
prior = F.softmax(torch.ones_like(b).float(), dim=1)
prior


tensor([[0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000]])

In [40]:
cross_entropy = nn.CrossEntropyLoss(reduction='none')
cross_entropy(prior, torch.argmax(b, dim=1))

tensor([1.6094, 1.6094, 1.6094, 1.6094, 1.6094, 1.6094, 1.6094, 1.6094, 1.6094,
        1.6094, 1.6094, 1.6094, 1.6094, 1.6094, 1.6094])

In [50]:
r_s = torch.randn(a.shape)
m = nn.Sigmoid()
r_s = m(r_s)

In [67]:
r_s.shape

torch.Size([15, 3, 84, 84])

In [78]:
BCE = -(a/256 * torch.log(r_s) + (1 - a/256) * torch.log(1 - r_s))
BCE

tensor([[[[0.8964, 1.3776, 1.3968,  ..., 0.5393, 0.5039, 0.8013],
          [0.3896, 1.0126, 0.1623,  ..., 0.7525, 0.7026, 0.1423],
          [0.2898, 2.6294, 1.6210,  ..., 1.0025, 0.8157, 0.4748],
          ...,
          [1.0197, 0.3582, 0.9278,  ..., 0.2167, 0.5153, 0.5956],
          [1.3958, 0.4942, 0.3381,  ..., 0.5057, 0.6814, 1.8045],
          [0.6241, 0.3327, 0.5814,  ..., 0.6714, 0.7805, 0.4100]],

         [[0.5437, 0.7469, 1.6851,  ..., 0.7264, 0.6496, 0.2133],
          [0.8684, 0.5148, 0.8361,  ..., 1.2772, 1.3730, 0.8977],
          [0.5045, 0.5200, 0.9974,  ..., 1.1619, 0.5084, 0.2228],
          ...,
          [0.4560, 1.1664, 0.4945,  ..., 0.5591, 0.8939, 0.6995],
          [1.7425, 0.7466, 0.5008,  ..., 0.6835, 0.4680, 1.3841],
          [0.5263, 0.5986, 0.8558,  ..., 0.3519, 0.7603, 0.5786]],

         [[1.2357, 0.7024, 1.1838,  ..., 1.1810, 0.3435, 0.8327],
          [0.2351, 0.4920, 0.5594,  ..., 0.1887, 0.3621, 0.2419],
          [0.6331, 0.6183, 1.1877,  ..., 0

In [80]:
loss = nn.BCELoss(reduction='none')
BCE = loss(r_s, (a/256))
BCE.view(r_s.shape[0], -1).mean(dim=1).mean()

tensor(0.8075)

In [58]:
loss = nn.BCELoss()
BCE = loss(r_s, (a/256))
BCE

tensor(0.8075)

In [63]:
def kl_div(mus, log_vars):
    return - 0.5 * (1 + log_vars - mus**2 - torch.exp(log_vars)).sum(dim=1)

kld = kl_div(mu_s, log_var_s)

In [66]:
kld

tensor([3.6877, 1.8342, 3.3309, 1.1454, 1.5170, 1.0895, 1.6448, 0.8020, 3.0228,
        1.8053, 4.7988, 4.8425, 1.9420, 1.6121, 1.4942],
       grad_fn=<MulBackward0>)

In [83]:
r_q = torch.randn(d.shape)
m = nn.Sigmoid()
r_q = m(r_q)

In [85]:
r_q.shape

torch.Size([250, 3, 84, 84])

In [88]:
reconstruction_loss = nn.BCELoss(reduction='none')
ce_loss_q = torch.nn.CrossEntropyLoss(reduction='none')

# BCE = loss(r_q, d/256)
# BCE.view(r_q.shape[0], -1).mean(dim=-1).shape

In [91]:
l = -reconstruction_loss(r_q, d).view(d.shape[0], -1).mean(dim=1) - ce_loss_q(
        F.softmax(torch.ones_like(e).float(), dim=1), torch.argmax(e, dim=1)) - kl_div(mu_q, log_var_q)
l.shape

torch.Size([250])

In [120]:
l

tensor([ -2.9386,  -4.2036,  -4.5291,  -3.3997,  -3.4342,  -4.0794,  -4.1871,
         -4.4376,  -4.2197,  -4.0751,  -3.5086,  -3.0915,  -3.9436,  -4.0142,
         -4.6801,  -4.5906,  -5.8111,  -5.2481,  -6.2033,  -5.0098,  -4.0412,
         -3.5116,  -3.8603,  -3.3279,  -3.7606,  -3.8182,  -5.6941,  -3.4970,
         -3.0530,  -4.1521,  -5.2496,  -4.9337,  -4.0076,  -3.9477,  -4.2349,
         -4.6654,  -3.1890,  -5.1358,  -3.7509,  -4.4481,  -3.4389,  -4.2766,
         -3.9453,  -3.8763,  -3.7239,  -3.0148,  -3.2896,  -3.0357,  -3.8589,
         -3.7308,  -3.1677,  -3.4604,  -4.3223,  -4.1827,  -4.1109, -43.0757,
        -40.6057, -42.4956, -41.6685, -41.9128,  -3.7205,  -4.9787,  -4.2097,
         -4.6712,  -3.1462,  -2.3690,  -3.0369,  -3.9227,  -3.8489,  -3.8800,
         -5.3113,  -2.9105,  -4.0527,  -5.0898,  -4.9180,  -2.1841,  -2.5850,
         -2.9370,  -3.9071,  -2.1923,  -4.7662,  -3.1652,  -4.4739,  -4.6347,
         -3.8818,  -2.7097,  -3.9221,  -3.2599,  -3.9838,  -4.53

In [142]:
l.reshape([50, 5]).t().shape

torch.Size([5, 50])

In [141]:
F.softmax(logits_q, dim=1)[::5,].shape

torch.Size([50, 5])

In [150]:
torch.mm(F.softmax(logits_q, dim=1)[::5,], l.reshape([50, 5]).t()).diag().shape

torch.Size([50])

In [152]:
lq = F.softmax(logits_q, dim=1)[::5,]
h = -torch.sum(torch.mul(lq, torch.log(lq + 1e-8)), dim=1)
h.shape

torch.Size([50])

In [151]:
a = torch.Tensor([[1,2,3], [4,5,6]])
b = torch.Tensor([[0,5,2], [8,9,7]])
torch.mul(a,b)

tensor([[ 0., 10.,  6.],
        [32., 45., 42.]])

/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn/src/zoo/lpo_utils.py:128: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(torch.ones_like(y_support).float()), torch.argmax(y_support, dim=1)) - kl_div(support_mu, support_log_var)  # = -L(x_s, y_s)
/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn/src/zoo/lpo_utils.py:131: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  F.softmax(torch.ones_like(y_queries).float()), torch.argmax(y_queries, dim=1)) - kl_div(queries_mu, queries_log_var)  # = -L(x_q, y_q)


RuntimeError: multi-target not supported at /opt/conda/conda-bld/pytorch_1616554793803/work/aten/src/THCUNN/generic/ClassNLLCriterion.cu:15